# get validation, n downsample indices

In [1]:
import json
import os
import gc
import numpy as np
import pandas as pd

def downsampling(positive_idx, negative_idx, ratio):
    idx = np.random.choice(negative_idx, int(ratio*len(negative_idx)))
    idx = np.concatenate((idx, positive_idx))
    return np.sort(idx).astype(int).tolist()


PATH1 = '/home/kai/data/kaggle/talkingdata/data/'
PATH0 = '/home/kai/data/kaggle/talkingdata/haoyandata/'
PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'

file = {}
train = pd.read_feather(PATH0 + 'train_cleaned.ftr')
# train = pd.read_csv(PATH1 + 'train_cleaned.csv', nrows=10000)
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
file['val_idx'] = val_idx[val_idx].index.values.astype(int).tolist()


# Downsampling
train = train[~val_idx]
negative = list(train[train['is_attributed']==0].index.values)
positive = list(train[train['is_attributed']==1].index.values)
ratio = len(positive)/len(negative)
idx_list = []
for i in range(5):
    idx_list.append(downsampling(positive, negative, ratio))
    print(i)
file['downsampling'] = idx_list
with open(PATH + 'idx_file.json', 'w') as f:
    json.dump(file, f)
print(ratio)
print('done')

0
1
2
3
4
done


In [3]:
print(ratio)

0.00246100785192494


# split files into several by validation and down sample indices

In [1]:
import os
import gc
import json
import pandas as pd

ORIGINAL_FILE_PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'
DOWNSAMPLING_PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
with open(DOWNSAMPLING_PATH + 'idx_file.json', 'r') as f:
    data = json.load(f)
    val_idx = data['val_idx']
    downsampling_idx = data['downsampling']

print('making directory')
directory = os.path.dirname((DOWNSAMPLING_PATH + 'val/'))
if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(len(downsampling_idx)):
    directory = os.path.dirname((DOWNSAMPLING_PATH + str(i) + '/'))
    if not os.path.exists(directory):
        os.makedirs(directory)
    
print('down sampling')
for file in os.listdir(ORIGINAL_FILE_PATH):
    if file.split('.')[-1] == suffix and file.split('.')[0].split('__')[0] == 'train':
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print(file)
        gc.collect()

making directory
down sampling
train__nunique_machine_channel.ftr
train__unique_ratio_machine_channel.ftr
train__range_count_ip_dayhourminute10.ftr
train__woe_ip_app_device_os.ftr
train__cumcount_ratio_ip_day.ftr
train__woe_ip_nextClickLeakDayFlt.ftr
train__woe_device_channel_nextClickLeakDayFlt.ftr
train__woe_os_channel_nextClickLeakDayFlt.ftr
train__woe_app_nextClickLeakDayFlt.ftr
train__cumcount_ip_day.ftr
train__woe_ip_os.ftr
train__woe_ip_os_channel.ftr
train__woe_ip_channel_nextClickLeakDayFlt.ftr
train__count_ip_device_os_day_hourminute10.ftr
train__unique_ratio_day_ip_app.ftr
train__range_count_ip_day_hour.ftr
train__range_count_ip_day.ftr
train__count_ip_app_device_os_day_hour.ftr
train__woe_device_nextClickLeakDayFlt.ftr
train__woe_ip_device_os.ftr
train__unique_ratio_day_ip_os.ftr
train__nunique_machine_app.ftr
train__woe_ip_channel.ftr
train__nunique_day_ip_channel.ftr
train__woe_ip_device_nextClickLeakDayFlt.ftr
train__count_ip_app_device_channel_day.ftr
train__count_ratio

In [7]:
from tqdm import tqdm
print('down sampling')
for file in tqdm(os.listdir(ORIGINAL_FILE_PATH)):
    if file.split('.')[-1] == suffix and file.split('.')[0] == 'train_cleaned':
        print(file)
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print('done')
        gc.collect()


  0%|          | 0/198 [00:00<?, ?it/s]

down sampling
train_cleaned.ftr



Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/kai/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/kai/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 198/198 [01:31<00:00,  2.16it/s]

done


# concatenate val, down sampling files into one

In [10]:
import os
import gc
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
dir_list = ['val/', '1/', '2/', '3/', '4/', '0/']
for x in dir_list:
    CURRENT_PATH = PATH + x
    file_name = PATH + x[:-1] + '.' + suffix
    df = pd.read_feather(CURRENT_PATH + 'train_cleaned.ftr')
    for y in tqdm(os.listdir(CURRENT_PATH)):
        splitted_y = y.split('.')
        if splitted_y[0] != 'train_cleaned' and splitted_y[1] == suffix:
            tmp = pd.read_feather(CURRENT_PATH + y)
            df[splitted_y[0].split('__')[1]] = tmp
            del tmp
    print('converting')
    df.to_feather(file_name)
    del df
    gc.collect()
    print(file_name)


 34%|███▍      | 34/100 [00:08<00:16,  3.99it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/kai/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/kai/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 100/100 [00:59<00:00,  1.69it/s]


converting


  0%|          | 0/100 [00:00<?, ?it/s]

/home/kai/data/kaggle/talkingdata/downsampling/val.ftr


100%|██████████| 100/100 [00:03<00:00, 26.99it/s]


...

converting


  0%|          | 0/100 [00:00<?, ?it/s]

/home/kai/data/kaggle/talkingdata/downsampling/4.ftr


100%|██████████| 100/100 [00:02<00:00, 45.18it/s]


converting
/home/kai/data/kaggle/talkingdata/downsampling/0.ftr


# test concate

In [ ]:
gc.collect()

325

In [1]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'
file_name = '/home/kai/data/kaggle/talkingdata/downsampling/test_cleaned.ftr'
df = pd.read_feather(PATH + 'test_cleaned.ftr')
for x in os.listdir(PATH):
    splitted = x.split('__')
    if splitted[0] == 'test_supplement':
        print(x)
        tmp = pd.read_feather(PATH + x)
        df[splitted[1].split('.')[0]] = tmp
        del tmp
        gc.collect()
print('converting')
df.to_feather(file_name)
del df
gc.collect()
print(file_name)

test_supplement__nunique_day_ip_app.ftr
test_supplement__woe_ip_channel_nextClickLeakDayFlt.ftr
test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
test_supplement__variance_ip_device_hour.ftr
test_supplement__woe_ip_app.ftr
test_supplement__count_ip_app_os_channel_day.ftr
test_supplement__woe_ip_nextClickLeakDayFlt.ftr
test_supplement__com1_ip.ftr
test_supplement__unique_ratio_day_ip_machine.ftr
test_supplement__unique_ratio_day_ip_device.ftr
test_supplement__woe_app_channel.ftr
test_supplement__woe_ip_app_device_os.ftr
test_supplement__nunique_day_ip_machine.ftr
test_supplement__count_ratio_channel_app.ftr
test_supplement__woe_os_channel_nextClickLeakDayFlt.ftr
test_supplement__range_count_ip_channel_dayhourminute.ftr
test_supplement__woe_app_device_channel.ftr
test_supplement__woe_os_nextClickLeakDayFlt.ftr
test_supplement__woe_channel.ftr
test_supplement__count_ip.ftr
test_supplement__woe_app_device.ftr
test_supplement__woe_app_os.ftr
test_supplement__woe_app_device_os.ftr
test

# name change: train(test_supplement)__feature_name.ftr

In [3]:
import os

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'

for file in os.listdir(PATH):
    tmp = file.split('.')
    if tmp[0] != 'train_cleaned' and tmp[0] != 'test_cleaned' and tmp[1] == 'ftr':
        ttmp = tmp[0].split('_')
        if ttmp[0] == 'train':
            nfile = 'train__' + '_'.join(ttmp[1:]) + '.ftr'
        else:
            nfile = '_'.join(ttmp[:2]) + '__' + '_'.join(ttmp[2:]) + '.ftr'
        os.rename(PATH + file, PATH + nfile)
        print(file, '------>', nfile)

train_count_app_device_day_hour.ftr train__count_app_device_day_hour.ftr
test_supplement_woe_app_os.ftr test_supplement__woe_app_os.ftr
test_supplement_woe_device.ftr test_supplement__woe_device.ftr
train_woe_ip_os_nextClickLeakDayFlt.ftr train__woe_ip_os_nextClickLeakDayFlt.ftr
test_supplement_woe_app_channel.ftr test_supplement__woe_app_channel.ftr
train_rang_count_ip_device_os_dayhourminute.ftr train__rang_count_ip_device_os_dayhourminute.ftr
test_supplement_woe_channel.ftr test_supplement__woe_channel.ftr
train_rang_count_ip_channel_dayhourminute.ftr train__rang_count_ip_channel_dayhourminute.ftr
train_count_ip_app_device_os_day_hour.ftr train__count_ip_app_device_os_day_hour.ftr
train_rang_count_ip_channel_dayhourminute10.ftr train__rang_count_ip_channel_dayhourminute10.ftr
train_variance_ip_device_hour.ftr train__variance_ip_device_hour.ftr
test_supplement_rang_count_ip_day.ftr test_supplement__rang_count_ip_day.ftr
test_supplement_rang_count_ip_hour.ftr test_supplement__rang_cou

# delete all test_supplement files in downsampling

In [14]:
import os

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
dir_list = ['val/','1/','2/','3/','4/','0/']
for x in dir_list:
    for y in os.listdir(PATH + x):
        if y.split('__')[0] == 'test_supplement':
            os.remove(PATH + x + y)
            print(PATH + x + y)

/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__nunique_day_ip_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_channel_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__variance_ip_device_hour.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__count_ip_app_os_channel_day.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__com1_ip.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_app_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_app_device_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_

/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__nunique_machine_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_app_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__nunique_day_ip_device.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_device_os_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ip_device_os_day_hourminute10.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ip_app_os_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ratio_ip_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_device.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_app_device_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/2/test_supplement__nunique_day_ip_app.ftr


In [1]:
import os
import pandas as pd
import gc
PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'

train = pd.read_feather(PATH + 'train_cleaned.ftr')
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
idx = ~val_idx
# val = train[val_idx]
train = train[idx]
print(gc.collect())

print('start')
for x in os.listdir(PATH):
    tmp = x.split('.')
    if tmp[-1] == 'ftr':
        ttmp = tmp[0].split('__')
        if ttmp[0] == 'train':
            f = pd.read_feather(PATH + x)
#             val[ttmp[1]] = f[val_idx]
            train[ttmp[1]] = f[idx]
            del f
            gc.collect()
            print(x, ttmp[1])
# val.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/downsampling/val_cleaned.ftr')

51
start
train__nunique_machine_channel.ftr nunique_machine_channel
train__unique_ratio_machine_channel.ftr unique_ratio_machine_channel
train__range_count_ip_dayhourminute10.ftr range_count_ip_dayhourminute10
train__woe_ip_app_device_os.ftr woe_ip_app_device_os
train__cumcount_ratio_ip_day.ftr cumcount_ratio_ip_day
train__woe_ip_nextClickLeakDayFlt.ftr woe_ip_nextClickLeakDayFlt
train__woe_device_channel_nextClickLeakDayFlt.ftr woe_device_channel_nextClickLeakDayFlt
train__woe_os_channel_nextClickLeakDayFlt.ftr woe_os_channel_nextClickLeakDayFlt
train__woe_app_nextClickLeakDayFlt.ftr woe_app_nextClickLeakDayFlt
train__cumcount_ip_day.ftr cumcount_ip_day
train__woe_ip_os.ftr woe_ip_os
train__woe_ip_os_channel.ftr woe_ip_os_channel
train__woe_ip_channel_nextClickLeakDayFlt.ftr woe_ip_channel_nextClickLeakDayFlt
train__count_ip_device_os_day_hourminute10.ftr count_ip_device_os_day_hourminute10
train__unique_ratio_day_ip_app.ftr unique_ratio_day_ip_app
train__range_count_ip_day_hour.ftr r

KeyboardInterrupt: 

In [ ]:
train.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/downsampling/train_cleaned.ftr')